In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
%matplotlib inline
#%matplotlib notebook
sns.set(rc={'figure.figsize':(11.7,8.27)})
from scipy import stats
import statsmodels.api as sm
from scipy.stats import ks_2samp
import scipy.stats as stats
from scipy.stats import ttest_rel
from datetime import datetime

In [2]:
df_16_20_long = pd.read_pickle(r'./otherData/demand_data_long_format.pkl')

In [4]:
df_16_20_long.head()

,Distributor,matCode,siv_demand
timestamp,,,
2016-01-01,A K ENTERPRISES,120082,0.00
2016-01-01,A K ENTERPRISES,120088,0.00
2016-01-01,A K ENTERPRISES,120089,0.00
2016-01-01,A K ENTERPRISES,120182,0.00
2016-01-01,A K ENTERPRISES,120229,0.00


In [5]:
df_16_20_long.index.max()

Timestamp('2020-02-01 00:00:00')

In [8]:
df_16_20_long['qtr']=df_16_20_long.index.quarter
df_16_20_long['year'] = df_16_20_long.index.year

In [11]:
df_16_20_long.groupby(['Distributor','year'])['matCode'].size().unstack().reset_index()

year,Distributor,2016,2017,2018,2019,2020
0,A K ENTERPRISES,8820,8820,8820,8820,1470
1,ACURA INDUSTRIAL LUBRICANTS,8820,8820,8820,8820,1470
2,AKILAN AGENCIES,8820,8820,8820,8820,1470
3,ANAND MOTORS,8820,8820,8820,8820,1470
4,AUTO WORLD,8820,8820,8820,8820,1470
5,AUTOMATION AND GENERAL ELECTRIC CO,8820,8820,8820,8820,1470
6,AWESH AUTOMOTIVE,8820,8820,8820,8820,1470
7,Aashvi Traders,8820,8820,8820,8820,1470
8,Adila Enterprise,8820,8820,8820,8820,1470
9,Agra Diesels & Automobiles,8820,8820,8820,8820,1470


In [12]:
data_org = pd.read_excel(r'D:\\raghav demand forecasting\\otherdata works\\REL_DATA_2.xlsx',sheetname='prep_data')

In [17]:
data_org.to_pickle('org_data_2016_18.pkl')

In [14]:
data_org.head()

,Distributor,STRIPES Code,Fiscal year/period,year,Date,Sum of Volume (L)
0,A K ENTERPRISES,135988,APR 2018,2018,2018-04-28,31.98
1,A K ENTERPRISES,136071,APR 2018,2018,2018-04-28,96.10
2,A K ENTERPRISES,139312,JUL 2018,2018,2018-07-28,416.39
3,A K ENTERPRISES,139343,SEP 2018,2018,2018-09-28,416.39
4,A K ENTERPRISES,139362,APR 2018,2018,2018-04-28,208.12


In [22]:
count_of_mats_year_wise = data_org.groupby(['Distributor','year'])['STRIPES Code'].nunique().unstack().reset_index()

In [23]:
count_of_mats_year_wise.fillna(0,inplace=True)

In [24]:
count_of_mats_year_wise

year,Distributor,2016,2017,2018
0,A K ENTERPRISES,0.00,20.00,93.00
1,ANAND MOTORS,107.00,124.00,125.00
2,AUTO WORLD,0.00,7.00,117.00
3,AUTOMATION AND GENERAL ELECTRIC CO,0.00,68.00,76.00
4,AWESH AUTOMOTIVE,0.00,19.00,62.00
5,Agra Diesels & Automobiles,0.00,0.00,14.00
6,Allahabad Auto Stores,0.00,0.00,29.00
7,Amit Automobiles,0.00,0.00,66.00
8,Apollo Traders,0.00,99.00,117.00
9,Arun Bansal & Company,37.00,32.00,22.00


In [28]:
df_19_20_long = pd.read_pickle(r'df_19_20_long.pkl')

In [33]:
df_19_20_long.head()

,Distributor,Material,Month of Effective Date,EMAPS Forecast (Ltrs),ICB,SIV,SOV
0,A K ENTERPRISES,123331,2019-01-01,0,0.00,0.00,0.00
1,A K ENTERPRISES,123331,2019-02-01,0,0.00,0.00,0.00
2,A K ENTERPRISES,123331,2019-03-01,0,0.00,0.00,0.00
3,A K ENTERPRISES,123331,2019-04-01,0,0.00,0.00,0.00
4,A K ENTERPRISES,123331,2019-05-01,0,0.00,0.00,0.00


In [34]:
df_19_20_long['qtr'] =  df_19_20_long['Month of Effective Date'].dt.quarter
df_19_20_long['year'] = df_19_20_long['Month of Effective Date'].dt.year

In [35]:
count_of_mats_2019 = df_19_20_long.groupby(['Distributor','year'])['Material'].nunique().unstack().reset_index()

In [36]:
count_of_mats_2019

year,Distributor,2019,2020
0,A K ENTERPRISES,140,140
1,ACURA INDUSTRIAL LUBRICANTS,129,129
2,AKILAN AGENCIES,73,73
3,ANAND MOTORS,203,203
4,AUTO WORLD,154,154
5,AUTOMATION AND GENERAL ELECTRIC CO,108,108
6,AWESH AUTOMOTIVE,62,62
7,Aashvi Traders,58,58
8,Adila Enterprise,37,37
9,Agra Diesels & Automobiles,13,13


In [37]:
df_19_20_long.head()

,Distributor,Material,Month of Effective Date,EMAPS Forecast (Ltrs),ICB,SIV,SOV,qtr,year
0,A K ENTERPRISES,123331,2019-01-01,0,0.00,0.00,0.00,1,2019
1,A K ENTERPRISES,123331,2019-02-01,0,0.00,0.00,0.00,1,2019
2,A K ENTERPRISES,123331,2019-03-01,0,0.00,0.00,0.00,1,2019
3,A K ENTERPRISES,123331,2019-04-01,0,0.00,0.00,0.00,2,2019
4,A K ENTERPRISES,123331,2019-05-01,0,0.00,0.00,0.00,2,2019


In [38]:
qtr_agg = df_19_20_long.groupby(['Distributor','Material','qtr','year'])['SIV'].sum().reset_index()

In [39]:
qtr_agg.head()

,Distributor,Material,qtr,year,SIV
0,A K ENTERPRISES,123331,1,2019,0.00
1,A K ENTERPRISES,123331,1,2020,20.00
2,A K ENTERPRISES,123331,2,2019,0.00
3,A K ENTERPRISES,123331,3,2019,0.00
4,A K ENTERPRISES,123331,4,2019,0.00


In [44]:
qtr_agg.groupby(['Distributor','Material','qtr','year'])['SIV'].sum().unstack('qtr').reset_index()

qtr,Distributor,Material,year,1,2,3,4
0,A K ENTERPRISES,123331,2019,0.00,0.00,0.00,0.00
1,A K ENTERPRISES,123331,2020,20.00,nan,nan,nan
2,A K ENTERPRISES,130393,2019,0.00,0.00,0.00,20.00
3,A K ENTERPRISES,130393,2020,0.00,nan,nan,nan
4,A K ENTERPRISES,132568,2019,0.00,0.00,0.00,17.59
5,A K ENTERPRISES,132568,2020,0.00,nan,nan,nan
6,A K ENTERPRISES,135371,2019,0.00,0.00,0.00,18.39
7,A K ENTERPRISES,135371,2020,0.00,nan,nan,nan
8,A K ENTERPRISES,135402,2019,0.00,0.00,0.00,0.00
9,A K ENTERPRISES,135402,2020,0.00,nan,nan,nan


In [45]:
qtr_agg.groupby(['Distributor','Material','qtr','year'])['SIV'].sum().unstack(['qtr','year']).reset_index()

qtr         Distributor Material      1             2      3      4
year                               2019   2020   2019   2019   2019
0       A K ENTERPRISES   123331   0.00  20.00   0.00   0.00   0.00
1       A K ENTERPRISES   130393   0.00   0.00   0.00   0.00  20.00
2       A K ENTERPRISES   132568   0.00   0.00   0.00   0.00  17.59
3       A K ENTERPRISES   135371   0.00   0.00   0.00   0.00  18.39
4       A K ENTERPRISES   135402   0.00   0.00   0.00   0.00   0.00
5       A K ENTERPRISES   136071   0.00  12.00  72.00  24.00  24.00
6       A K ENTERPRISES   139286   0.00   0.00   0.00 208.00   0.00
7       A K ENTERPRISES   139312   0.00   0.00   0.00   0.00   0.00
8       A K ENTERPRISES   139314   0.00   0.00 208.00   0.00   0.00
9       A K ENTERPRISES   139341   0.00   0.00 208.00   0.00   0.00
10      A K ENTERPRISES   139343   0.00   0.00   0.00   0.00   0.00
11      A K ENTERPRISES   139362   0.00 208.00   0.00   0.00   0.00
12      A K ENTERPRISES   139385   0.00   0.00   0.00   0.00   0.00
13      A K ENTERPRISES   139386   0.00   0.00   0.00   0.00   0.00
14      A K ENTERPRISES   139445   0.00   0.00   0.00   0.00   0.00
15      A K ENTERPRISES   139448   0.00   0.00 100.00   0.00   0.00
16      A K ENTERPRISES   139449   0.00   0.00 250.00   0.00   0.00
17      A K ENTERPRISES   139451   0.00   0.00 262.50   0.00   0.00
18      A K ENTERPRISES   139452   0.00   0.00  40.00   0.00   0.00
19      A K ENTERPRISES   139453   0.00   0.00   0.00   0.00   0.00
20      A K ENTERPRISES   139457   0.00   0.00   0.00   0.00   0.00
21      A K ENTERPRISES   139464 640.00   0.00   0.00 100.00   0.00
22      A K ENTERPRISES   139465 900.00   0.00 540.00   0.00   0.00
23      A K ENTERPRISES   139467   0.00 100.00   0.00   0.00  40.00
24      A K ENTERPRISES   139468   0.00   0.00   0.00   0.00   0.00
25      A K ENTERPRISES   139470   0.00 200.00 200.00 200.00 200.00
26      A K ENTERPRISES   139471   0.00   0.00   0.00   0.00   0.00
27      A K ENTERPRISES   139472   0.00   0.00   0.00  60.00 120.00
28      A K ENTERPRISES   139475   0.00 100.00   0.00   0.00 100.00
29      A K ENTERPRISES   139478   0.00   0.00   0.00   0.00   0.00
...                 ...      ...    ...    ...    ...    ...    ...
20137  Zorin Enterprise   140471   0.00   0.00   0.00   0.00  40.00
20138  Zorin Enterprise   140472   0.00 210.00   0.00   0.00  28.00
20139  Zorin Enterprise   140473   0.00 270.00   0.00   0.00  36.00
20140  Zorin Enterprise   140578   0.00   0.00   0.00   0.00 200.00
20141  Zorin Enterprise   140579   0.00   0.00   0.00   0.00  70.00
20142  Zorin Enterprise   140581   0.00   0.00   0.00   0.00 200.00
20143  Zorin Enterprise   141229   0.00  70.00   0.00   0.00   0.00
20144  Zorin Enterprise   141381   0.00 100.00   0.00   0.00 100.00
20145  Zorin Enterprise   141390   0.00  75.00   0.00   0.00   0.00
20146  Zorin Enterprise   141391   0.00 100.00   0.00   0.00   0.00
20147  Zorin Enterprise   141394   0.00   0.00   0.00   0.00  50.00
20148  Zorin Enterprise   141395   0.00   0.00   0.00   0.00  75.00
20149  Zorin Enterprise   141399   0.00 672.00   0.00   0.00  48.00
20150  Zorin Enterprise   141409   0.00   0.00   0.00   0.00  21.60
20151  Zorin Enterprise   141410   0.00   0.00   0.00   0.00  24.00
20152  Zorin Enterprise   141413   0.00 480.00   0.00   0.00  24.00
20153  Zorin Enterprise   141421   0.00   0.00   0.00   0.00 152.42
20154  Zorin Enterprise   141422   0.00  76.21   0.00   0.00  76.21
20155  Zorin Enterprise   141425   0.00   0.00   0.00   0.00  63.51
20156  Zorin Enterprise   141427   0.00 571.58   0.00   0.00  45.73
20157  Zorin Enterprise   141457   0.00 240.00   0.00   0.00   0.00
20158  Zorin Enterprise   141661   0.00 120.00   0.00   0.00 120.00
20159  Zorin Enterprise   141662   0.00 120.00   0.00   0.00  90.00
20160  Zorin Enterprise   141979   0.00 200.00   0.00   0.00 800.00
20161  Zorin Enterprise   141980   0.00 900.00   0.00   0.00 360.00
20162  Zorin Enterprise   141993   0.00 750.00   0.00   

In [51]:
qtr_agg['qtr_weight'] = qtr_agg.SIV*100/qtr_agg.groupby(['Distributor','qtr','year'])\
                        ['SIV'].transform('sum')

In [52]:
qtr_agg.head()

,Distributor,Material,qtr,year,SIV,qtr_weight
0,A K ENTERPRISES,123331,1,2019,0.00,0.00
1,A K ENTERPRISES,123331,1,2020,20.00,0.16
2,A K ENTERPRISES,123331,2,2019,0.00,0.00
3,A K ENTERPRISES,123331,3,2019,0.00,0.00
4,A K ENTERPRISES,123331,4,2019,0.00,0.00


In [59]:
qtr_agg['qtr_w_rank'] =qtr_agg.groupby(['Distributor','qtr','year'])['qtr_weight'].rank(ascending=False,method='dense')

In [60]:
qtr_agg.head()

,Distributor,Material,qtr,year,SIV,qtr_weight,qtr_w_rank
0,A K ENTERPRISES,123331,1,2019,0.00,0.00,29.00
1,A K ENTERPRISES,123331,1,2020,20.00,0.16,27.00
2,A K ENTERPRISES,123331,2,2019,0.00,0.00,41.00
3,A K ENTERPRISES,123331,3,2019,0.00,0.00,43.00
4,A K ENTERPRISES,123331,4,2019,0.00,0.00,44.00


In [61]:
qtr_agg.to_excel(r'dist_mat_qtr_weight.xlsx',index=False)

In [62]:
qtr_agg.shape

(100835, 7)

In [70]:
qtr_agg_wide_fmt = qtr_agg.groupby(['Distributor','Material','qtr','year'])['qtr_weight'].mean().unstack(['qtr','year']).reset_index()

In [71]:
qtr_agg_wide_fmt.fillna(0,inplace=True)

In [72]:
qtr_agg_wide_fmt.columns = [''.join(str(col)).strip() for col in qtr_agg_wide_fmt.columns.values]

In [73]:
qtr_agg_wide_fmt.head()

,"('Distributor', '')","('Material', '')","(1, 2019)","(1, 2020)","(2, 2019)","(3, 2019)","(4, 2019)"
0,A K ENTERPRISES,123331,0.00,0.16,0.00,0.00,0.00
1,A K ENTERPRISES,130393,0.00,0.00,0.00,0.00,0.10
2,A K ENTERPRISES,132568,0.00,0.00,0.00,0.00,0.08
3,A K ENTERPRISES,135371,0.00,0.00,0.00,0.00,0.09
4,A K ENTERPRISES,135402,0.00,0.00,0.00,0.00,0.00


In [74]:
qtr_agg_wide_fmt.to_excel(r'dist_mat_qtr_weight_wide_fmt.xlsx',index=False)

In [44]:
qtr_agg.groupby(['Distributor','Material','qtr','year'])['SIV'].sum().unstack('qtr').reset_index()

qtr,Distributor,Material,year,1,2,3,4
0,A K ENTERPRISES,123331,2019,0.00,0.00,0.00,0.00
1,A K ENTERPRISES,123331,2020,20.00,nan,nan,nan
2,A K ENTERPRISES,130393,2019,0.00,0.00,0.00,20.00
3,A K ENTERPRISES,130393,2020,0.00,nan,nan,nan
4,A K ENTERPRISES,132568,2019,0.00,0.00,0.00,17.59
5,A K ENTERPRISES,132568,2020,0.00,nan,nan,nan
6,A K ENTERPRISES,135371,2019,0.00,0.00,0.00,18.39
7,A K ENTERPRISES,135371,2020,0.00,nan,nan,nan
8,A K ENTERPRISES,135402,2019,0.00,0.00,0.00,0.00
9,A K ENTERPRISES,135402,2020,0.00,nan,nan,nan


In [80]:
qtr_agg_all_data = df_16_20_long.groupby(['Distributor','matCode','qtr','year'])['siv_demand'].sum().reset_index()

qtr_agg_all_data['qtr_weight'] = qtr_agg_all_data.siv_demand*100/qtr_agg_all_data.groupby(['Distributor','qtr','year'])\
                        ['siv_demand'].transform('sum')

qtr_agg_all_data['qtr_w_rank'] =qtr_agg_all_data.groupby(['Distributor','qtr','year'])['qtr_weight'].rank(ascending=False,method='dense')
qtr_agg_all_data_wide = qtr_agg_all_data.groupby(['Distributor','matCode','qtr','year'])['qtr_w_rank'].mean().unstack(['qtr','year']).reset_index()
qtr_agg_all_data_wide.fillna(0,inplace=True)

In [88]:
qtr_agg_all_data_wide.head()

qtr       Distributor matCode    1                           2             \
year                          2016 2017  2018  2019  2020 2016 2017  2018   
0     A K ENTERPRISES  120082 0.00 0.00 24.00 29.00 30.00 0.00 0.00 30.00   
1     A K ENTERPRISES  120088 0.00 0.00 24.00 29.00 30.00 0.00 0.00 30.00   
2     A K ENTERPRISES  120089 0.00 0.00 24.00 29.00 30.00 0.00 0.00 30.00   
3     A K ENTERPRISES  120182 0.00 0.00 24.00 29.00 30.00 0.00 0.00 30.00   
4     A K ENTERPRISES  120229 0.00 0.00 24.00 29.00 30.00 0.00 0.00 30.00   

qtr           3                     4                    
year  2019 2016 2017  2018  2019 2016  2017  2018  2019  
0    41.00 0.00 0.00 26.00 43.00 0.00 12.00 37.00 44.00  
1    41.00 0.00 0.00 26.00 43.00 0.00 12.00 37.00 44.00  
2    41.00 0.00 0.00 26.00 43.00 0.00 12.00 37.00 44.00  
3    41.00 0.00 0.00 26.00 43.00 0.00 12.00 37.00 44.00  
4    41.00 0.00 0.00 26.00 43.00 0.00 12.00 37.00 44.00

In [89]:
qtr_agg_all_data_wide.columns

MultiIndex(levels=[[1, 2, 3, 4, 'matCode', 'Distributor'], [2016, 2017, 2018, 2019, 2020, '']],
           labels=[[5, 4, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3], [5, 5, 0, 1, 2, 3, 4, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]],
           names=['qtr', 'year'])

In [90]:
qtr_agg_all_data.to_excel('qtr_agg_all_data_long.xlsx',index=False)

In [91]:
qtr_agg_all_data_wide.shape

(125686, 19)